In [1]:

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
import json
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches

In [2]:
data_path_train = r'C:\Users\Steven\OneDrive - Cal State LA\Documents\train_metadata.json'
with open(data_path_train) as json_file:
    meta_train = json.load(json_file)
    
    
#loading json file for test data
data_path_test = r'C:\Users\Steven\OneDrive - Cal State LA\Documents\test_metadata.json'
with open(data_path_test) as json_file:
    meta_test = json.load(json_file) 

In [3]:
#Finding keys in train dictionary
meta_train.keys()

dict_keys(['annotations', 'images', 'categories', 'genera', 'institutions', 'distances', 'license'])

dict_keys(['annotations', 'images', 'categories', 'genera', 'institutions', 'distances', 'license'])

In [ ]:
#creating seperate dataframes from metadata
annotations_train =  pd.json_normalize(meta_train ['annotations'])
categories_train =  pd.json_normalize(meta_train ['categories'])
images_train =  pd.json_normalize(meta_train ['images'])
genera_train =  pd.json_normalize(meta_train ['genera'])
distance_train =  pd.json_normalize(meta_train ['distances'])
licenses_train =  pd.json_normalize(meta_train ['license'])
institutions_train =  pd.json_normalize(meta_train ['institutions'])

In [ ]:
del genera_train
del licenses_train
del institutions_train

In [ ]:
#Looking at test set
df_test = pd.DataFrame(meta_test)

#creating test data
df_test = df_test.drop(['license'], axis=1)

print(df_test['file_name'])
# adding file path
df_test = df_test[['image_id','file_name']]
df_test['file_path']='C:/Users/Steven/OneDrive - Cal State LA/Documents/image/'+df_test['file_name']


In [ ]:
# Complete df
df_merge = pd.merge(images_train[['image_id','file_name']],annotations_train[['genus_id','category_id','image_id']] , on='image_id')
df_merge = pd.merge(df_merge[['genus_id','image_id','file_name','category_id']],categories_train[['category_id','scientificName','family','genus','species']] , on='category_id')
df_merge['file_path']='C:/Users/Steven/OneDrive - Cal State LA/Documents/images2/'+df_merge['file_name']
df_merge['name']=df_merge['genus']+' '+df_merge['species']
df_train = df_merge[['category_id','genus_id','image_id','family','genus','species','name','file_name','file_path']]

df_train.head()

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.io import imread, imshow
from skimage.transform import rescale, resize, downscale_local_mean
y=[]
def makeimage(i2,img2):
    image=mpimg.imread(df_train['file_path'][i2])
    #img=imread(df_train['file_path'][i2],as_gray=True)
    #img=img[0:665,0:595]
    #img2.append(np.reshape(img, (665*595)))
    y.append(df_train['category_id'][i2])
    feature_matrix = np.zeros((665,595) )
    feature_matrix.shape
    
    for i in range(0,665):
        for j in range(0,595):
            feature_matrix[i][j] = ((int(image[i,j,0]) + int(image[i,j,1]) + int(image[i,j,2]))/3)
    
    img2.append(np.reshape(feature_matrix, (665*595))) 
    #features.shape

    #print(features)


i2=0
img2=[]
while i2<500:
    makeimage(i2,img2)
    i2=i2+1


In [ ]:


img2=np.array(img2)
print(np.array(img2).shape)

In [ ]:

dff=pd.DataFrame(y,columns = ['Name'])
y=dff['Name']
print(y)
from sklearn import preprocessing
X=img2
#c
X=preprocessing.scale(X)


In [ ]:
seed= 1
yes=np.random.seed(seed)

In [ ]:
print(yes)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=111)
print(X_train)
print(y_train)
print(y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
# for getting accuracy of our classifiers
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

#my_decisiontree = DecisionTreeClassifier(random_state=3)       
#my_decisiontree.fit(X_train, y_train)
#y_predict = my_decisiontree.predict(X_test)


#my_random_forest= RandomForestClassifier(n_estimators=100, bootstrap=True, n_jobs=-1, random_state=111)
#my_random_forest.fit(X_train, y_train)
#y_predict =my_random_forest.predict(X_test)

#my_knn_for_cs4661 = KNeighborsClassifier(n_neighbors=4) 
#my_knn_for_cs4661.fit(X_train, y_train)
#y_predict = my_knn_for_cs4661.predict(X_test)
#score_lr = accuracy_score(y_test, y_predict)
#print(score_lr)
#model = MLPClassifier( random_state=111, hidden_layer_sizes=(10,),learning_rate_init = 0.05, solver='adam', alpha=1, verbose=True, activation='logistic')
from keras.models import Sequential
from  keras.wrappers.scikit_learn  import  KerasClassifier
from  sklearn.model_selection  import  GridSearchCV
# import the core layers:
from keras.layers import Dense, Dropout, Activation, Flatten

def model_creator():

    # define:

    

    # design the structure:

    ## Designing the ANN Structure (with 784 inputs, 10 outputs and 100 neuron in a hidden layer):
    # Declare Sequential model to build our network:
    model = Sequential()
    input_size = 665*595
    hidden_neurons = 100
    out_size = 10
    # -----------------------------------------
    # first layer: input layer
    # Input layer does not do any processing, so no need to define the input layer in this problem.

    # -----------------------------------------
    # second layer: hidden layer:
    model.add(Dense(hidden_neurons, input_dim = input_size))  # Nuerons
    model.add(Activation('sigmoid')) # Activation

    # -----------------------------------------
    # third layer: output layer:
    model.add(Dense(out_size, input_dim = hidden_neurons))  # Nuerons
    model.add(Activation('softmax')) # Activation

    # compile:

    model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')

    # return: 

    return model 

seed= 2
np.random.seed(seed)
model = KerasClassifier(build_fn = model_creator, verbose=2)
batch_size = [30 , 50 , 100 ] 
epochs = [10 , 15 , 20]

param_grid = dict(batch_size=batch_size, epochs=epochs)

grid=GridSearchCV(estimator=model,param_grid=param_grid,scoring='neg_log_loss',cv=10)
fitted_model = grid.fit(X_train, y_train)
print("Best: %f using %s" % (fitted_model.best_score_, fitted_model.best_params_))


In [ ]:


fitted_model2 = model.fit(X_train, y_train)

# Prediction:
model.predict(X_test)
model.score(X_test,y_test)
#cm=confusion_matrix(y_predict, y_test)
#print('The accuracy is: ', accuracy(cm))

In [ ]:
print(X_train.shape)

In [ ]:

from  sklearn.model_selection  import  GridSearchCV

seed= 1
np.random.seed(seed)



neurons = [] 

i=5

while i<=250:
    neurons.append(i)
    i=i+5
print(neurons)


lst_tuple = [x for x in zip(*[iter(neurons)])]
print(lst_tuple)



from sklearn.preprocessing import StandardScaler
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.pipeline import Pipeline
GRID = [
    {
     'estimator': [MLPClassifier(random_state=seed)],
     'estimator__solver': ['adam'],
     'estimator__learning_rate_init': [0.02],
     'estimator__hidden_layer_sizes': lst_tuple,
     'estimator__activation': ['logistic'],
     'estimator__alpha': [1]
     }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', MLPClassifier())])

grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, 
                            scoring='accuracy', cv=10, verbose=True, 
                            )

grid_search.fit(X, y)






In [ ]:
print("Best: %f using %s" % (grid_search.best_score_, grid_search.best_params_))